This notebook I did topic modeling with each review's TF-IDF instead of count vectors

In [ ]:
import gensim
import numpy as np
import pandas as pd
import nltk
import os
from nltk.corpus import stopwords
import pickle
import re
from sklearn.decomposition import TruncatedSVD
from textblob import TextBlob

In [ ]:
observation = pd.read_csv('tokenized_text')

In [ ]:
observation = pd.read_csv('user_info_subset')

In [ ]:
observation.iloc[0]['tokens']

In [ ]:
nltk.download('punkt')

stopword = set(stopwords.words('english'))

In [ ]:
#Adding custom stopwords
stopword = stopword.union(set(['food', 'this', 'place', 'the', 'of', 'is', 'came', 'was', 'for', 'have', 'had'
                           ,'and', 'get', 'one', 'food', 'guy','?','!','place', 'good', 'fries','burger', 'burgers',
                            'got', 'eat','great', 'us', 'asked', 'service', 'back', 'time', 'like', 'vegas', 'go',
                            'try', 'animal', 'style', 'double', 'good', 'just', 'always', 'location', 'fresh',
                              'east', 'coast', 'order', 'ordered', 'fast']))

In [ ]:
#Returns tokens for each review
def  chapter_reader():
    for i in observation['text']:
        yield (x for x in 
            gensim.utils.tokenize(i, lowercase=True, deacc=True, 
                                  errors="ignore")
            if (x not in stopword) and len(x) > 1)

In [ ]:
corpus = []
dictionary = gensim.corpora.Dictionary(chapter_reader())
dictionary.filter_extremes(no_below=1, no_above=0.8, keep_n=100000)
for values in chapter_reader():
    corpus.append(dictionary.doc2bow(values))

In [ ]:
corpus

In [ ]:
#Prints most frequent word for each review
for i,vector in enumerate(corpus):
    try:
        most_index, most_count = max(vector, key=lambda item: item[1])
    except ValueError:
        continue
    print( "Review " + str(i+1) + " most used word: ",end='')
    print( dictionary[most_index], most_count)

In [ ]:
#Converts tokens to their term TF-IDF
tfidf = gensim.models.TfidfModel(corpus, normalize=True)
corpus_tfidf = tfidf[corpus]
# lsi_tfidf = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
# lsi_tfidf.print_topics()

In [ ]:
#LDA modeling to create topics and print them.
lda = gensim.models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=3)
lda.print_topics()

In [ ]:
observation.columns

In [ ]:
#Extract document topic probabilities and save to csv with length of review and usefulness as columns
# def extract_topic_prob():
doc_topics = []
probs = []
for i in chapter_reader():
    bow = dictionary.doc2bow(i)
    doc_topics.append(lda.get_document_topics(bow))
for i in doc_topics:
    a = []
    for j in i:
        a.append(j[1])
    probs.append(a)
df_doc_probs = pd.DataFrame(probs)
df_doc_probs = df_doc_probs.fillna(0)
df_doc_probs['length'] = observation['length']
df_doc_probs['useful'] = observation['useful']
df_doc_probs.to_csv('lda_doc_topic_probs')

In [ ]:
"""Trying to add user data as columns and saving to csv"""
# df_doc_probs['review_count'] = observation['review_count']
# df_doc_probs['account_age'] = observation['account_age']
# df_doc_probs['useful_sum'] = observation['useful_sum']
# df_doc_probs['stars'] = observation['stars']
# df_doc_probs['funny_sum'] = observation['funny_sum']
# df_doc_probs['compliment_cool'] = observation['compliment_cool']
# df_doc_probs.to_csv('topic_probs_desperate')

In [ ]:
len(doc_topics)

In [ ]:
len(probs)

In [ ]:
df_doc_probs.isnull().values.any()

In [ ]:
df_doc_probs.head()

In [ ]:
#Tried LSI just to see if topics would be similar
lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=3)
lsi.print_topics()